In [1]:
''
import os
import copy
import random
import numpy as np
import pretty_midi as pyd
from loader.dataloader import MIDI_Loader

s_dir = "" # folder address
midi_path = "/Users/wxn/Desktop/surf/Music-SketchNet-master/data/IrishFolkSong/session/sessiontune1.mid"

In [2]:
# load data from Midis, because bpm = 120，so one beat time = 60 / 120 = 0.5
# And in 4/4 we divide 4 beat to 24 step/frames, each will be 0.5 * 4 / 24  = 0.5 / 6 sec
# It takes a little bit long time.
ml = MIDI_Loader("Irish",minStep = 0.5 / 6)
ml.load_single_midi(midi_path)

Dataset Name: Irish
start to load mid from /Users/wxn/Desktop/surf/Music-SketchNet-master/data/IrishFolkSong/session/sessiontune1.mid


In [3]:
# process all files to the mesaure data for VAE tranining
data = ml.processed_all()

start process Irish Folk Song dataset
start to get notes
find ahead notes in Irish Folk Song dataset in /Users/wxn/Desktop/surf/Music-SketchNet-master/data/IrishFolkSong/session/sessiontune1
get notes in 1 files
calc notes success! 1 files in total
processing succeed


In [4]:
print(data[0])

{'name': '/Users/wxn/Desktop/surf/Music-SketchNet-master/data/IrishFolkSong/session/sessiontune1', 'raw': <pretty_midi.pretty_midi.PrettyMIDI object at 0x7fabd870cdd0>, 'notes': [74, 128, 128, 62, 128, 128, 62, 128, 62, 128, 62, 128, 128, 69, 128, 128, 62, 128, 128, 62, 128, 62, 128, 62, 128, 128, 74, 128, 128, 62, 128, 128, 62, 128, 62, 128, 62, 128, 128, 72, 128, 128, 128, 128, 128, 71, 128, 128, 72, 128, 128, 74, 128, 128, 62, 128, 128, 62, 128, 62, 128, 62, 128, 128, 69, 128, 128, 67, 128, 128, 66, 128, 128, 69, 128, 128, 67, 128, 128, 64, 128, 128, 60, 128, 128, 64, 128, 128, 67, 128, 128, 69, 128, 128, 71, 128, 128, 72, 128, 128, 74, 128, 128, 62, 128, 128, 62, 128, 62, 128, 62, 128, 128, 69, 128, 128, 62, 128, 128, 62, 128, 62, 128, 62, 128, 128, 74, 128, 128, 62, 128, 128, 62, 128, 62, 128, 62, 128, 128, 72, 128, 128, 128, 128, 128, 71, 128, 128, 72, 128, 128, 74, 128, 128, 71, 128, 128, 71, 128, 128, 69, 128, 128, 71, 128, 128, 74, 128, 128, 72, 128, 128, 69, 128, 128, 67, 128

In [5]:
for d in data:
    del d['raw']
    print(d)
    print(len(d))

print(data)

np.save("data/irish_single.npy", data)

{'name': '/Users/wxn/Desktop/surf/Music-SketchNet-master/data/IrishFolkSong/session/sessiontune1', 'notes': [74, 128, 128, 62, 128, 128, 62, 128, 62, 128, 62, 128, 128, 69, 128, 128, 62, 128, 128, 62, 128, 62, 128, 62, 128, 128, 74, 128, 128, 62, 128, 128, 62, 128, 62, 128, 62, 128, 128, 72, 128, 128, 128, 128, 128, 71, 128, 128, 72, 128, 128, 74, 128, 128, 62, 128, 128, 62, 128, 62, 128, 62, 128, 128, 69, 128, 128, 67, 128, 128, 66, 128, 128, 69, 128, 128, 67, 128, 128, 64, 128, 128, 60, 128, 128, 64, 128, 128, 67, 128, 128, 69, 128, 128, 71, 128, 128, 72, 128, 128, 74, 128, 128, 62, 128, 128, 62, 128, 62, 128, 62, 128, 128, 69, 128, 128, 62, 128, 128, 62, 128, 62, 128, 62, 128, 128, 74, 128, 128, 62, 128, 128, 62, 128, 62, 128, 62, 128, 128, 72, 128, 128, 128, 128, 128, 71, 128, 128, 72, 128, 128, 74, 128, 128, 71, 128, 128, 71, 128, 128, 69, 128, 128, 71, 128, 128, 74, 128, 128, 72, 128, 128, 69, 128, 128, 67, 128, 128, 64, 128, 128, 60, 128, 128, 64, 128, 128, 67, 128, 128, 69, 128

In [6]:
# Process the midi

In [7]:
# note extraction
hold_state = 128
rest_state = 129
def extract_note(x, pad_token = 128):
    d = []
    for i in x:
        if i < 128:
            d.append(i)
    ori_d = len(d)
    d.extend([pad_token] * (len(x) - len(d)))
    return np.array(d), ori_d

def extract_rhythm(x, hold_token = 2, rest_token = 3):
    d = []
    for i in x:
        if i < 128:
             d.append(1)
        elif i == hold_state:
             d.append(hold_token)
        else:
             d.append(rest_token)
    return np.array(d)


In [8]:
# process rhythm and pitch tokens
split_size = 24
new_data = []
# change here to be train_x/validate_x/test_x
for i,d in enumerate(data):
    d = np.array(d["notes"])
    ds = np.split(d, list(range(split_size,len(d),split_size)))
    data = []
    for sd in ds:
        if len(sd) != split_size:
            continue
        q,k = extract_note(sd)
        if k == 0:
            continue
        s = extract_rhythm(sd)
        data.append([sd,q,s,k])
    new_data.append(data)
    if i % 1000 == 0:
        print("processed:", i)

processed: 0


In [9]:
print(new_data)

[[[array([ 74, 128, 128,  62, 128, 128,  62, 128,  62, 128,  62, 128, 128,
        69, 128, 128,  62, 128, 128,  62, 128,  62, 128,  62]), array([ 74,  62,  62,  62,  62,  69,  62,  62,  62,  62, 128, 128, 128,
       128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128]), array([1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 1,
       2, 1]), 10], [array([128, 128,  74, 128, 128,  62, 128, 128,  62, 128,  62, 128,  62,
       128, 128,  72, 128, 128, 128, 128, 128,  71, 128, 128]), array([ 74,  62,  62,  62,  62,  72,  71, 128, 128, 128, 128, 128, 128,
       128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128]), array([2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1,
       2, 2]), 7], [array([ 72, 128, 128,  74, 128, 128,  62, 128, 128,  62, 128,  62, 128,
        62, 128, 128,  69, 128, 128,  67, 128, 128,  66, 128]), array([ 72,  74,  62,  62,  62,  62,  69,  67,  66, 128, 128, 128, 128,
       128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 1

In [10]:
# we need to extract each measure in each song
final_data = []
print(len(new_data))
for d in new_data:
    for dd in d:
        final_data.append(dd)
print(len(final_data))
print(final_data)
np.save("data/irish_processed.npy", final_data)

1
71
[[array([ 74, 128, 128,  62, 128, 128,  62, 128,  62, 128,  62, 128, 128,
        69, 128, 128,  62, 128, 128,  62, 128,  62, 128,  62]), array([ 74,  62,  62,  62,  62,  69,  62,  62,  62,  62, 128, 128, 128,
       128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128]), array([1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 1,
       2, 1]), 10], [array([128, 128,  74, 128, 128,  62, 128, 128,  62, 128,  62, 128,  62,
       128, 128,  72, 128, 128, 128, 128, 128,  71, 128, 128]), array([ 74,  62,  62,  62,  62,  72,  71, 128, 128, 128, 128, 128, 128,
       128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128]), array([2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1,
       2, 2]), 7], [array([ 72, 128, 128,  74, 128, 128,  62, 128, 128,  62, 128,  62, 128,
        62, 128, 128,  69, 128, 128,  67, 128, 128,  66, 128]), array([ 72,  74,  62,  62,  62,  62,  69,  67,  66, 128, 128, 128, 128,
       128, 128, 128, 128, 128, 128, 128, 128, 128, 12

/Users/wxn/opt/anaconda3/envs/torch/lib/python3.7/site-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [11]:
'''
In this file we eval the sketchnet
Notice the eval of stage-1 and stage-2 is the same
So we include as one file
We provide a sceneario for simply sketching
You can choose one another song B and take any part of it as a sketch information to the original song A, to see what the generation will be
'''
import torch
import os
import numpy as np
from torch import optim
from torch.nn import functional as F
from SketchVAE.sketchvae import SketchVAE
from torch import optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.distributions import Normal
from SketchNet.sketchnet import SketchNet
from utils.helpers import *
import time
###############################
# initial parameters
s_dir = ""
zp_dims = 128
zr_dims = 128
pf_dims = 512
gen_dims = 1024
combine_dims = 512
combine_head = 4
combine_num = 4
pf_num = 2
inpaint_len = 4
seq_len = 16
total_len = 16
batch_size = 1
n_epochs = 15
data_path = [
    s_dir + "data/irish-dis-measure-vae-train-whole.npy",
    s_dir + "data/irish-dis-measure-vae-validate-whole.npy",
    s_dir + "data/irish-dis-measure-vae-validate-repetition.npy",
    s_dir + "data/irish-dis-measure-vae-validate-non-repetition.npy",
    s_dir + "data/irish_processed.npy"
]
lr = 1e-4
decay = 0.9999
##############################
##############  for vae init ##############
vae_hidden_dims = 1024
vae_zp_dims = 128
vae_zr_dims = 128
vae_beta = 0.1
vae_input_dims = 130
vae_pitch_dims = 129
vae_rhythm_dims = 3
vae_seq_len = 6 * 4
vae_beat_num = 4
vae_tick_num = 6
############################

In [12]:
# input data dis-measure-vae
def processed_data_tensor(data):
    print("processed data:")
    gd = []
    px = []
    rx = []
    len_x = []
    nrx = []
    total = 0
    for i, d in enumerate(data):
        gd.append([list(dd[0]) for dd in d])
        px.append([list(dd[1]) for dd in d])
        rx.append([list(dd[2]) for dd in d])
        len_x.append([dd[3] for dd in d])
        if len(gd[-1][-1]) != vae_seq_len:
            gd[-1][-1].extend([128] * (vae_seq_len - len(gd[-1][-1])))
            px[-1][-1].extend([128] * (vae_seq_len - len(px[-1][-1])))
            rx[-1][-1].extend([2] * (vae_seq_len - len(rx[-1][-1])))
    for i,d in enumerate(len_x):
        for j,dd in enumerate(d):
            if len_x[i][j] == 0:
                gd[i][j][0] = 60
                px[i][j][0] = 60
                rx[i][j][0] = 1
                len_x[i][j] = 1
                total += 1
    gd = np.array(gd)
    px = np.array(px)
    rx = np.array(rx)
    len_x = np.array(len_x)
    for d in rx:
        nnrx = []
        for dd in d:
            temp = np.zeros((vae_seq_len, vae_rhythm_dims))
            lins = np.arange(0, len(dd))
            temp[lins, dd - 1] = 1
            nnrx.append(temp)
        nrx.append(nnrx)
    nrx = np.array(nrx)
    gd = torch.from_numpy(gd).long()
    px = torch.from_numpy(px).long()
    rx = torch.from_numpy(rx).float()
    len_x = torch.from_numpy(len_x).long()
    nrx = torch.from_numpy(nrx).float()
    print("processed finish! zeros:", total)
    print(gd.size(),px.size(),rx.size(),len_x.size(),nrx.size())
    return TensorDataset(px, rx, len_x, nrx, gd)

In [13]:
# validate_set = np.load(data_path[4],allow_pickle = True)
validate_set = [data]
print(validate_set)
print(validate_set[0][0])
print(len(validate_set[0][1]))
validate_loader = DataLoader(
    dataset = processed_data_tensor(validate_set),
    batch_size = 1,
    shuffle = False,
    num_workers = 8,
    pin_memory = True,
    drop_last = False
)
# dataset = processed_data_tensor(validate_set)
# print(validate_set)

validate_data = []
for i,d in enumerate(validate_loader):
    validate_data.append(d)
print(len(validate_data))

[[[array([ 74, 128, 128,  62, 128, 128,  62, 128,  62, 128,  62, 128, 128,
        69, 128, 128,  62, 128, 128,  62, 128,  62, 128,  62]), array([ 74,  62,  62,  62,  62,  69,  62,  62,  62,  62, 128, 128, 128,
       128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128]), array([1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 1,
       2, 1]), 10], [array([128, 128,  74, 128, 128,  62, 128, 128,  62, 128,  62, 128,  62,
       128, 128,  72, 128, 128, 128, 128, 128,  71, 128, 128]), array([ 74,  62,  62,  62,  62,  72,  71, 128, 128, 128, 128, 128, 128,
       128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128]), array([2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1,
       2, 2]), 7], [array([ 72, 128, 128,  74, 128, 128,  62, 128, 128,  62, 128,  62, 128,
        62, 128, 128,  69, 128, 128,  67, 128, 128,  66, 128]), array([ 72,  74,  62,  62,  62,  62,  69,  67,  66, 128, 128, 128, 128,
       128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 1

In [14]:
# load VAE model
vae_model = SketchVAE(
    vae_input_dims, vae_pitch_dims, vae_rhythm_dims, vae_hidden_dims,
    vae_zp_dims, vae_zr_dims, vae_seq_len, vae_beat_num, vae_tick_num, 4000)
dic = torch.load("model_backup/sketchvae-loss_0.04306925100494333_acc_0.9972101456969356_epoch_26_it_174824.pt")

for name in list(dic.keys()):
    dic[name.replace('module.', '')] = dic.pop(name)
vae_model.load_state_dict(dic)

if torch.cuda.is_available():
    print('Using: ', torch.cuda.get_device_name(torch.cuda.current_device()))
    vae_model.cuda()
else:
    print('Using: CPU')
vae_model.eval()
print(vae_model.training)

Using: CPU
False


In [15]:
# import model
save_path = s_dir +"model_backup/"
save_period = 5

# think about traning with mse
model = SketchNet(
    zp_dims, zr_dims,
    pf_dims, gen_dims, combine_dims,
    pf_num, combine_num, combine_head,
    inpaint_len, total_len,
    vae_model, True
)
dic = torch.load(save_path + "sketchNet-stage-1loss_0.5944014993628857_acc_0.8705233683628317_epoch_90_it_111870.pt")
for name in list(dic.keys()):
    dic[name.replace('module.', '')] = dic.pop(name)
model.load_state_dict(dic)
model.set_stage("sketch")

if torch.cuda.is_available():
    print('Using: ', torch.cuda.get_device_name(torch.cuda.current_device()))
    model.cuda()
else:
    print('Using: CPU')
# print(model)
# model.eval()

Using: CPU


In [16]:
# for normal eval process
def process_raw_x(raw_x, n_past, n_inpaint, n_future):
    raw_px, raw_rx, raw_len_x, raw_nrx, raw_gd = raw_x
    past_px = raw_px[:,:n_past,:]
    inpaint_px = raw_px[:,n_past:n_past + n_inpaint,:]
    future_px = raw_px[:,n_future:,:]
    past_rx = raw_rx[:,:n_past,:]
    inpaint_rx = raw_rx[:,n_past:n_past + n_inpaint,:]
    future_rx = raw_rx[:,n_future:,:]
    past_len_x = raw_len_x[:,:n_past]
    inpaint_len_x = raw_len_x[:,n_past:n_past + n_inpaint]
    future_len_x = raw_len_x[:,n_future:]
    past_nrx = raw_nrx[:,:n_past,:]
    inpaint_nrx = raw_nrx[:,n_past:n_past + n_inpaint,:]
    future_nrx = raw_nrx[:,n_future:,:]
    past_gd = raw_gd[:,:n_past,:]
    inpaint_gd = raw_gd[:,n_past:n_past + n_inpaint,:]
    future_gd = raw_gd[:,n_future:,:]
    re = [
        past_px, past_rx, past_len_x, past_nrx, past_gd,
        inpaint_px, inpaint_rx, inpaint_len_x, inpaint_nrx, inpaint_gd,
        future_px, future_rx, future_len_x, future_nrx, future_gd,
    ]
    return re
def get_acc(recon, gd):
    recon = recon.cpu().detach().numpy()
    gd = gd.cpu().detach().numpy()
    return np.sum(recon == gd) / recon.size


In [ ]:
model.set_stage("sketch")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

save_period = 5
losses = []
step = 0
n_past = 6
n_future = 10
n_inpaint = 4
iteration = 0
output = []
v_mean_loss = 0.0
v_mean_acc = 0.0
total = 0
print(vae_model.training)
for i in range(1):
    val = []
    val.append(validate_data[i])
    v_raw_x = process_raw_x(validate_data[i], n_past, n_inpaint, n_future)
    for k in range(len(v_raw_x)):
        v_raw_x[k] = v_raw_x[k].to(device = device,non_blocking = True)
    v_past_px, v_past_rx, v_past_len_x, v_past_nrx, v_past_gd, \
    v_inpaint_px, v_inpaint_rx, v_inpaint_len_x, v_inpaint_nrx, v_inpaint_gd,\
    v_future_px, v_future_rx, v_future_len_x, v_future_nrx, v_future_gd = v_raw_x
    v_inpaint_gd_whole = v_inpaint_gd.contiguous().view(-1)
    v_past_x = [v_past_px, v_past_rx, v_past_len_x, v_past_nrx, v_past_gd]
    v_inpaint_x = [v_inpaint_px, v_inpaint_rx, v_inpaint_len_x, v_inpaint_nrx, v_inpaint_gd]
    v_future_x = [v_future_px, v_future_rx, v_future_len_x, v_future_nrx, v_future_gd]

    model.eval()
    with torch.no_grad():
        v_recon_x, _,_,_ = model(v_past_x, v_future_x, v_inpaint_x)
        v_loss = F.cross_entropy(v_recon_x.view(-1, v_recon_x.size(-1)), v_inpaint_gd_whole, reduction = "mean")
        v_acc = get_acc(v_recon_x.view(-1, v_recon_x.size(-1)).argmax(-1), v_inpaint_gd_whole)
        v_mean_loss += v_loss.item()
        v_mean_acc += v_acc
        v_result = v_recon_x.argmax(-1)
    total += 1
    output.append(
        {
         "past": v_past_gd.cpu().detach().numpy(),
         "future":v_future_gd.cpu().detach().numpy(),
         "inpaint":v_result.cpu().detach().numpy(),
         "gd":v_inpaint_gd.cpu().detach().numpy(),
         "acc":v_acc,
         "nll":v_loss.item()
        }
    )
    print(output)